# 2D fitness krajina

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display

# Fitness krajina (Rastrigin) a fitness = -Rastrigin (maximalizujeme)
def rastrigin(x, y):
    return 20 + (x**2 - 10*np.cos(2*np.pi*x)) + (y**2 - 10*np.cos(2*np.pi*y))

def fitness(pop_xy):
    x = pop_xy[:, 0]
    y = pop_xy[:, 1]
    return -rastrigin(x, y)

# Doména (typická pro Rastrigin)
bounds = np.array([[-5.12, 5.12],
                   [-5.12, 5.12]])

# Grid pro vizualizaci krajiny
grid_n = 220
xs = np.linspace(bounds[0, 0], bounds[0, 1], grid_n)
ys = np.linspace(bounds[1, 0], bounds[1, 1], grid_n)
X, Y = np.meshgrid(xs, ys)
Z = -r# igin(X, Y)


# # luce: truncation selection + Gaussian mutace + elitismus

rng = np.random.default_rng(0)

N = 80            # velikost populace
gens = 70         # počet generací
elite_frac = 0.2  # podíl elit
elite_n = max(1, int(N * elite_frac))

sigma0 = 0.45     # počáteční síla mutace
sigma_decay = 0.99  # mírné "ochlazování" mutace

# Inicializace populace uniformně v doméně
pop = np.column_stack([
    rng.uniform(bounds[0, 0], bounds[0, 1], N),
    rng.uniform(bounds[1, 0], bounds[1, 1], N),
])

history_pop = []
best_hist = []
avg_hist = []
best_xy_hist = []

sigma = sigma0
for g in range(gens):
    fit = fitness(pop)
    order = np.argsort(fit)[::-1]   # seřadit od nejlepšího
    pop = pop[order]
    fit = fit[order]

    history_pop.append(pop.copy())
    best_hist.append(fit[0])
    avg_hist.append(fit.mean())
    best_xy_hist.append(pop[0].copy())

    elites = pop[:elite_n]
    parents = elites[rng.integers(0, elite_n, size=N - elite_n)]
    offspring = parents + rng.normal(0, sigma, size=(N - elite_n, 2))

    # oříznutí do domény
    offspring[:, 0] = np.clip(offspring[:, 0], bounds[0, 0], bounds[0, 1])
    offspring[:, 1] = np.clip(offspring[:, 1], b# s[1, 0], bounds[1,#

    pop = np.vstack([elites, offspring])
    sigma *= sigma_decay

best_hist = np.array(best_hist)
avg_hist = np.array(avg_hist)
best_xy_hist = np.array(best_xy_hist)

best_overall_idx = int(np.argmax(best_hist))
best_overall_xy = best_xy_hist[best_overall_idx]
best_overall_fit = float(best_hist[best_overall_idx])
# # mace: kontury + populace + learning curve

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 5))

# Krajina
ax0.contourf(X, Y, Z, levels=35)
ax0.set_xlabel("x")
ax0.set_ylabel("y")

# Populace (scatter)
pop0 = history_pop[0]
sc = ax0.scatter(pop0[:, 0], pop0[:, 1], s=30)

# Learning curve
ax1.set_title("Průběh fitness")
ax1.set_xlabel("generace")
ax1.set_ylabel("fitness")
ax1.set_xlim(0, gens - 1)
ymin = min(best_hist.min(), avg_hist.min()) - 1.0
ymax = max(best_hist.max(), avg_hist.max()) + 1.0
ax1.set_ylim(ymin, ymax)

(line_best,) = ax1.plot([], [])
(line_avg,) = ax1.plot([], [])
ax1.legend(["best", "average"])

def init():
    line_best.set_data([], [])
    line_avg.set_data([], [])
    return sc, line_best, line_avg

def update(frame):
    popf = history_pop[frame]

    # Zvýraznit nejlepšího jedince v generaci větší velikostí bodu
    sizes = np.full(N, 30.0)
    sizes[0] = 140.0

    sc.set_offsets(popf)
    sc.set_sizes(sizes)

    xdata = np.arange(frame + 1)
    line_best.set_data(xdata, best_hist[:frame + 1])
    line_avg.set_data(xdata, avg_hist[:frame + 1])

    ax0.set_title(f"2D fitness krajina + populace (generace {frame})")
    return sc, line_best, line_avg

anim = animation.FuncAnimation(
    fig, update, frames=gens, init_func=init, interval=120, blit=True
)

plt.close(fig)
display(HTML(anim.to_jshtml()))


#  Souhrn (statický graf) + nejlepší bod

plt.figure(figsize=(6, 4))
plt.plot(best_hist, label="best")
plt.plot(avg_hist, label="average")
plt.xlabel("generace")
plt.ylabel("fitness")
plt.title("Learning curve (souhrn)")
plt.legend()
plt.show()

print("Nejlepší nalezený bod (x, y):", best_overall_xy)
print("Nejlepší fitness:", best_overall_fit)
print("Poznámka: optimum Rastrigin je v (0, 0) a maximální fitness je 0 (protože fitness = -Rastrigin).")
#

# Logistický neuron v 1D (jasná interpretace parametrů).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display


# Prostředí (1D): pozice s_t, cíl target, akce a_t ∈ {-1, +1}

target = 2.0      # cílová pozice
step = 0.2        # velikost kroku na akci
T = 50            # délka epizody v krocích

def sigmoid_stable(z):
    """Numericky stabilní sigmoid (kvůli velkým |z| při vykreslování)."""
    z = np.asarray(z, dtype=float)
    out = np.empty_like(z)
    pos = z >= 0
    out[pos] = 1.0 / (1.0 + np.exp(-z[pos]))
    expz = np.exp(z[~pos])
    out[~pos] = expz / (1.0 + expz)
    return out

def simulate_batch(theta, starts):
    """
    Simulace pro více startů najednou (vektorově).
    theta = (w, b) pro logistickou jednotku p = sigmoid(w*s + b).
    Politika je deterministická: akce +1 pokud (w*s + b) >= 0, jinak -1.
    Fitness = záporná průměrná absolutní vzdálenost od cíle v čase (a přes starty).
    """
    w, b = theta
    s = starts.astype(float).copy()
    acc = 0.0
    for _ in range(T):
        z = w * s + b
        a = np.where(z >= 0.0, 1.0, -1.0)
        s = s + step * a
        acc += np.abs(s - target)
    return - (acc / T).mean()

def simulate_trajectory(theta, s0):
    """Jedna trajektorie pro konkrétní start s0 (deterministická politika)."""
    w, b = theta
    s = float(s0)
    traj = [s]
    for _ in range(T):
        z = w * s + b
        a = 1.0 if z >= 0.0 else -1.0
        s = s + step * a
        traj.append(s)
    return np.array(traj)

def threshold(theta):
    """Rozhodovací práh v prostoru stavu: w*s + b = 0  =>  s = -b/w (pokud w != 0)."""
    w, b = theta
    if abs(w) < 1e-12:
        return np.nan
    return -b / w


#  Evoluce: truncation selection + elitismus + Gaussian mutace

rng = np.random.default_rng(0)

N = 120               # velikost populace
gens = 60             # počet generací
elite_frac = 0.2
elite_n = max(1, int(N * elite_frac))

sigma0 = 1.1          # počáteční síla mutace
sigma_decay = 0.985   # postupné snižování mutace
clip_val = 20.0       # ořez parametrů (w,b) kvůli extrémům

n_starts = 12         # počet startů pro evaluaci (společné pro celou populaci v generaci)
start_low, start_high = -4.0, 4.0

# Populace: každý jedinec je (w, b)
pop = rng.normal(0, 1, size=(N, 2))

pop_history = []
best_hist = []
avg_hist = []
best_theta_hist = []

best0 = None

sigma = sigma0
for g in range(gens):
    # Stejné starty pro všechny jedince (férové srovnání v generaci)
    starts = rng.uniform(start_low, start_high, size=n_starts)

    fits = np.array([simulate_batch(pop[i], starts) for i in range(N)])
    order = np.argsort(fits)[::-1]  # od nejlepšího
    pop = pop[order]
    fits = fits[order]

    if g == 0:
        best0 = pop[0].copy()

    pop_history.append(pop.copy())
    best_hist.append(fits[0])
    avg_hist.append(fits.mean())
    best_theta_hist.append(pop[0].copy())

    # Elity zůstávají
    elites = pop[:elite_n]

    # Rodiče se losují z elit
    parents = elites[rng.integers(0, elite_n, size=N - elite_n)]

    # Potomci: mutace
    offspring = parents + rng.normal(0, sigma, size=(N - elite_n, 2))
    offspring = np.clip(offspring, -clip_val, clip_val)

    pop = np.vstack([elites, offspring])
    sigma *= sigma_decay

pop_history = np.array(pop_history)          # (gens, N, 2)
best_hist = np.array(best_hist)
avg_hist = np.array(avg_hist)
best_theta_hist = np.array(best_theta_hist)

best_final = best_theta_hist[-1]
w0, b0 = best0
wf, bf = best_final

print("Nejlepší jedinec v generaci 0:   w = %.4f, b = %.4f, práh s* = %.4f" % (w0, b0, threshold(best0)))
print("Nejlepší jedinec na konci:      w = %.4f, b = %.4f, práh s* = %.4f" % (wf, bf, threshold(best_final)))
print("Cíl prostředí: target = %.4f" % target)
print("Poznámka k interpretaci: práh s* = -b/w určuje, kde se přepíná akce (+1 vs -1).")


#  Animace: (w,b) populace + learning curve (best/average)

w_all = pop_history[:, :, 0].ravel()
b_all = pop_history[:, :, 1].ravel()
wmin, wmax = w_all.min(), w_all.max()
bmin, bmax = b_all.min(), b_all.max()
pad_w = 0.10 * (wmax - wmin + 1e-9)
pad_b = 0.10 * (bmax - bmin + 1e-9)

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 5))

# Scatter populace v prostoru parametrů (w,b)
p0 = pop_history[0]
sc = ax0.scatter(p0[:, 0], p0[:, 1], s=25)
ax0.set_xlabel("w")
ax0.set_ylabel("b")
ax0.set_xlim(wmin - pad_w, wmax + pad_w)
ax0.set_ylim(bmin - pad_b, bmax + pad_b)

# Learning curve
ax1.set_title("Průběh fitness")
ax1.set_xlabel("generace")
ax1.set_ylabel("fitness")
ax1.set_xlim(0, gens - 1)
ymin = min(best_hist.min(), avg_hist.min()) - 0.05
ymax = max(best_hist.max(), avg_hist.max()) + 0.05
ax1.set_ylim(ymin, ymax)
(line_best,) = ax1.plot([], [])
(line_avg,) = ax1.plot([], [])
ax1.legend(["best", "average"])

def init():
    line_best.set_data([], [])
    line_avg.set_data([], [])
    return sc, line_best, line_avg

def update(frame):
    popf = pop_history[frame]
    sizes = np.full(N, 25.0)
    sizes[0] = 140.0  # nejlepší jedinec v generaci (po seřazení je na indexu 0)

    sc.set_offsets(popf)
    sc.set_sizes(sizes)

    x = np.arange(frame + 1)
    line_best.set_data(x, best_hist[:frame + 1])
    line_avg.set_data(x, avg_hist[:frame + 1])

    ax0.set_title(f"Populace parametrů (w,b) - generace {frame}")
    return sc, line_best, line_avg

anim = animation.FuncAnimation(fig, update, frames=gens, init_func=init, interval=120, blit=True)
plt.close(fig)
display(HTML(anim.to_jshtml()))


#  Statické vizualizace: (a) sigmoid a práh, (b) trajektorie, (c) křivky fitness


# (a) Sigmoidní křivka p(s) pro best na začátku vs na konci
s_grid = np.linspace(-5, 5, 600)
p_best0 = sigmoid_stable(w0 * s_grid + b0)
p_bestf = sigmoid_stable(wf * s_grid + bf)

plt.figure(figsize=(8, 4))
plt.plot(s_grid, p_best0, label="best (gen 0)")
plt.plot(s_grid, p_bestf, label="best (final)")
plt.axhline(0.5, linestyle="--")
plt.axvline(target, linestyle="--")
thr0 = threshold(best0)
thrf = threshold(best_final)
if np.isfinite(thr0):
    plt.axvline(thr0, linestyle=":")
if np.isfinite(thrf):
    plt.axvline(thrf, linestyle=":")
plt.title("Logistická jednotka: p(s) = sigmoid(w*s + b) a rozhodovací práh")
plt.xlabel("stav s")
plt.ylabel("p(s)")
plt.legend()
plt.show()

# (b) Trajektorie nejlepšího jedince (pro několik startů)
starts_demo = [-3.0, 0.0, 4.0]
plt.figure(figsize=(8, 4))
for s0 in starts_demo:
    traj = simulate_trajectory(best_final, s0)
    plt.plot(traj, label=f"s0 = {s0}")
plt.axhline(target, linestyle="--")
plt.title("Trajektorie nejlepší politiky (deterministické přepínání akce)")
plt.xlabel("čas t")
plt.ylabel("stav s_t")
plt.legend()
plt.show()

# (c) Learning curve (souhrn)
plt.figure(figsize=(8, 4))
plt.plot(best_hist, label="best")
plt.plot(avg_hist, label="average")
plt.title("Learning curve (souhrn)")
plt.xlabel("generace")
plt.ylabel("fitness")
plt.legend()
plt.show()


# 2D navigace: sparse vs shaping (obtížnost a signál).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display


# Prostředí: 2D navigace k cíli (bod v rovině)
#    Porovnání 2 odměn:
#    - sparse: 1 pokud agent dojde do cíle, jinak 0
#    - shaping: - (průměrná vzdálenost od cíle v čase)  -> hladký signál


# Cíl a parametry epizody
goal = np.array([0.0, 0.0])
step = 0.30         # maximální délka kroku za 1 akci
T = 30              # délka epizody
goal_r = 0.40       # poloměr "dosažení cíle"
obs_scale = 5.0     # škálování pozorování (aby hodnoty nebyly extrémní)

# Starty generujeme na kružnici / prstenci: agent není "u cíle zadarmo"
r_min, r_max = 3.5, 5.5

def sample_starts_ring(rng, n, r_min=r_min, r_max=r_max):
    angles = rng.uniform(0, 2*np.pi, size=n)
    radii = rng.uniform(r_min, r_max, size=n)
    return np.column_stack([radii*np.cos(angles), radii*np.sin(angles)])


#  Politika (model agenta): jednoduchá lineární mapa + tanh
#    Parametry (genom) mají délku 6:
#    - W je 2x2 (4 čísla)
#    - b je 2 (2 čísla)
#
#    Pozorování: o = (goal - pos) / obs_scale
#    Akce: a = tanh(W o + b), normalizace na max normu 1
#    Pohyb: pos <- pos + step * a


def unpack_params(theta):
    W = theta[:4].reshape(2, 2)
    b = theta[4:6]
    return W, b

def simulate_batch(theta, starts, reward_mode):
    """
    Vektorová simulace pro více epizod (různé starty) najednou.
    Vrací:
      - mean_fitness: průměrná fitness přes epizody
      - success_rate: podíl epizod, kde agent došel do cíle (pro diagnostiku)
    """
    W, b = unpack_params(theta)
    pos = starts.astype(float).copy()          # (E,2)
    E = pos.shape[0]
    reached = np.zeros(E, dtype=bool)
    sum_dist = np.zeros(E, dtype=float)        # pro shaping

    for _ in range(T):
        # vzdálenost od cíle na začátku kroku
        dist = np.linalg.norm(pos - goal, axis=1)
        not_done = ~reached

        # shaping: akumulujeme vzdálenost jen pro ty, kteří ještě neskončili
        if reward_mode == "shaping":
            sum_dist[not_done] += dist[not_done]

        # výpočet akce (jen pro not_done; pro reached je pohyb nulový)
        obs = (goal - pos) / obs_scale                         # (E,2)
        u = obs @ W.T + b                                      # (E,2)
        a = np.tanh(u)                                         # (E,2)
        an = np.linalg.norm(a, axis=1, keepdims=True)          # (E,1)
        a = a / np.maximum(1.0, an)                            # omezíme normu na <= 1

        pos_new = pos + (step * a) * not_done[:, None]         # pokud reached, nepohybujeme

        # kontrola dosažení cíle po pohybu
        dist_new = np.linalg.norm(pos_new - goal, axis=1)
        reached = reached | (dist_new < goal_r)

        pos = pos_new

    success_rate = reached.mean()

    if reward_mode == "sparse":
        # fitness je průměrný success (0/1) přes epizody
        mean_fitness = success_rate
    elif reward_mode == "shaping":
        # fitness je záporná průměrná vzdálenost (blíže k 0 je lepší)
        mean_fitness = (-sum_dist / T).mean()
    else:
        raise ValueError("reward_mode musí být 'sparse' nebo 'shaping'.")

    return mean_fitness, success_rate

def simulate_trajectory(theta, start):
    """Jedna trajektorie pro vykreslení (vrací posloupnost pozic)."""
    W, b = unpack_params(theta)
    pos = np.array(start, dtype=float)
    traj = [pos.copy()]
    for _ in range(T):
        if np.linalg.norm(pos - goal) < goal_r:
            break
        obs = (goal - pos) / obs_scale
        u = W @ obs + b
        a = np.tanh(u)
        an = np.linalg.norm(a)
        if an > 1.0:
            a = a / an
        pos = pos + step * a
        traj.append(pos.copy())
    return np.array(traj)


#  Evoluce: truncation selection + elitismus + Gaussian mutace


def run_evolution(reward_mode, seed=0,
                  N=120, gens=60, elite_frac=0.2,
                  sigma0=0.8, sigma_decay=0.99,
                  starts_n=8, clip_val=10.0):
    rng = np.random.default_rng(seed)

    # populace parametrů (N,6)
    pop = rng.normal(0, 1, size=(N, 6))

    elite_n = max(1, int(N * elite_frac))
    sigma = sigma0

    best_hist = []
    avg_hist = []
    best_params_hist = []
    best_succ_hist = []

    for g in range(gens):
        # stejné starty pro všechny jedince v generaci (férové srovnání)
        starts = sample_starts_ring(rng, starts_n)

        fits = np.empty(N, dtype=float)
        succs = np.empty(N, dtype=float)
        for i in range(N):
            f, srate = simulate_batch(pop[i], starts, reward_mode)
            fits[i] = f
            succs[i] = srate

        order = np.argsort(fits)[::-1]
        pop = pop[order]
        fits = fits[order]
        succs = succs[order]

        best_hist.append(fits[0])
        avg_hist.append(fits.mean())
        best_params_hist.append(pop[0].copy())
        best_succ_hist.append(succs[0])

        # elitismus
        elites = pop[:elite_n]

        # rodiče losujeme jen z elit
        parents = elites[rng.integers(0, elite_n, size=N - elite_n)]

        # mutace
        offspring = parents + rng.normal(0, sigma, size=(N - elite_n, 6))
        offspring = np.clip(offspring, -clip_val, clip_val)

        pop = np.vstack([elites, offspring])
        sigma *= sigma_decay

    return (np.array(best_hist),
            np.array(avg_hist),
            np.array(best_params_hist),
            np.array(best_succ_hist))

# Spusťte dva běhy se stejným seedem (pro porovnání)
N = 120
gens = 60
seed = 1

best_sparse, avg_sparse, bestP_sparse, succ_sparse = run_evolution("sparse",  seed=seed, N=N, gens=gens)
best_shaping, avg_shaping, bestP_shaping, succ_shaping = run_evolution("shaping", seed=seed, N=N, gens=gens)

print("Konečný nejlepší (sparse):  fitness = %.3f, success_rate (na tréninkových startech) = %.3f"
      % (best_sparse[-1], succ_sparse[-1]))
print("Konečný nejlepší (shaping): fitness = %.3f, success_rate (na tréninkových startech) = %.3f"
      % (best_shaping[-1], succ_shaping[-1]))
print("Poznámka: u sparse je fitness přímo success_rate, u shaping je fitness záporná průměrná vzdálenost (blíže k 0 je lepší).")


#  Animace: jak se zlepšuje trajektorie nejlepšího jedince v čase
#    (porovnání sparse vs shaping vedle sebe)


fixed_start = np.array([5.0, 0.0])  # pevný start pro vizualizaci zlepšování

fig, (axL, axR) = plt.subplots(1, 2, figsize=(12, 5))

def setup_ax(ax, title):
    ax.set_aspect("equal", adjustable="box")
    ax.set_xlim(-6, 6)
    ax.set_ylim(-6, 6)
    ax.set_title(title)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    # cíl a "cílová oblast"
    ax.scatter([goal[0]], [goal[1]], s=80)
    circle = plt.Circle((goal[0], goal[1]), goal_r, fill=False, linestyle="--")
    ax.add_patch(circle)

setup_ax(axL, "Sparse reward: trajektorie nejlepšího jedince")
setup_ax(axR, "Reward shaping: trajektorie nejlepšího jedince")

(lineL,) = axL.plot([], [], linewidth=2)
(lineR,) = axR.plot([], [], linewidth=2)

txtL = axL.text(-5.8, 5.4, "")
txtR = axR.text(-5.8, 5.4, "")

def init():
    lineL.set_data([], [])
    lineR.set_data([], [])
    txtL.set_text("")
    txtR.set_text("")
    return lineL, lineR, txtL, txtR

def update(frame):
    # sparse
    trL = simulate_trajectory(bestP_sparse[frame], fixed_start)
    lineL.set_data(trL[:, 0], trL[:, 1])
    txtL.set_text(f"gen {frame} | best = {best_sparse[frame]:.3f}")

    # shaping
    trR = simulate_trajectory(bestP_shaping[frame], fixed_start)
    lineR.set_data(trR[:, 0], trR[:, 1])
    txtR.set_text(f"gen {frame} | best = {best_shaping[frame]:.3f}")

    return lineL, lineR, txtL, txtR

anim = animation.FuncAnimation(fig, update, frames=gens, init_func=init, interval=140, blit=True)
plt.close(fig)
display(HTML(anim.to_jshtml()))


#  Learning curves: best a average (sparse vs shaping)


fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 4))

ax0.plot(best_sparse, label="best")
ax0.plot(avg_sparse, label="average")
ax0.set_title("Sparse reward: fitness v čase")
ax0.set_xlabel("generace")
ax0.set_ylabel("fitness (success_rate)")
ax0.set_ylim(-0.05, 1.05)
ax0.legend()

ax1.plot(best_shaping, label="best")
ax1.plot(avg_shaping, label="average")
ax1.set_title("Reward shaping: fitness v čase")
ax1.set_xlabel("generace")
ax1.set_ylabel("fitness (− průměrná vzdálenost)")
ax1.legend()

plt.show()


#  Finální srovnání trajektorií (více startů)


starts_demo = np.array([
    [ 5.0,  0.0],
    [ 0.0,  5.0],
    [-5.0,  0.0],
    [ 0.0, -5.0],
])

best_final_sparse = bestP_sparse[-1]
best_final_shaping = bestP_shaping[-1]

fig, (bx0, bx1) = plt.subplots(1, 2, figsize=(12, 5))

setup_ax(bx0, "Finální politika (sparse): trajektorie z více startů")
for s0 in starts_demo:
    tr = simulate_trajectory(best_final_sparse, s0)
    bx0.plot(tr[:, 0], tr[:, 1])
    bx0.scatter([s0[0]], [s0[1]], s=30)

setup_ax(bx1, "Finální politika (shaping): trajektorie z více startů")
for s0 in starts_demo:
    tr = simulate_trajectory(best_final_shaping, s0)
    bx1.plot(tr[:, 0], tr[:, 1])
    bx1.scatter([s0[0]], [s0[1]], s=30)

plt.show()


#  Diagnostika: success rate finálních politik na větším testu


rng_test = np.random.default_rng(123)
test_starts = sample_starts_ring(rng_test, 200)

_, sr_sparse = simulate_batch(best_final_sparse, test_starts, "sparse")
_, sr_shaping = simulate_batch(best_final_shaping, test_starts, "sparse")  # success měříme stejně pro obě

print("Test success_rate (200 startů na prstenci):")
print("  finální sparse  :", round(sr_sparse, 3))
print("  finální shaping :", round(sr_shaping, 3))


# Goodhart/exploit (proč je skóre kritické).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display


# GOODHART / EXPLOIT DEMO (2D navigace)
# Skutečný cíl úlohy: dojít do cíle (goal).
# Špatně navržené skóre (bad): odměňuje "stání v bonus zóně" (beacon),
# což vede k exploitu: agent farmí odměnu a ignoruje cíl.
# Opravené skóre (good): dává velký bonus za dosažení cíle + průběžné shaping
# a ukáže, že změna skóre zásadně změní chování.



# Prostředí a geometrie

goal = np.array([0.0, 0.0])
goal_r = 0.45

beacon = np.array([3.0, 3.0])   # "bonus zóna", která svádí k exploitu
beacon_r = 0.80

step = 0.28     # maximální krok za 1 akci (po normalizaci)
T = 55          # maximální délka epizody
obs_scale = 6.0 # škálování pozorování

# starty na prstenci, aby agent nezačínal u cíle
r_min, r_max = 4.5, 6.2

def sample_starts_ring(rng, n):
    ang = rng.uniform(0, 2*np.pi, size=n)
    rad = rng.uniform(r_min, r_max, size=n)
    return np.column_stack([rad*np.cos(ang), rad*np.sin(ang)])


#  Model politiky: lineární mapa + tanh + normalizace
# Pozorování: [ (goal-pos)/c , (beacon-pos)/c ]  (4D)
# Genom: W (2x4) + b (2) -> celkem 10 parametrů

def unpack(theta):
    W = theta[:8].reshape(2, 4)
    b = theta[8:10]
    return W, b

def policy_action(theta, pos):
    # pos: (E,2)
    W, b = unpack(theta)
    og = (goal - pos) / obs_scale
    ob = (beacon - pos) / obs_scale
    obs = np.concatenate([og, ob], axis=1)     # (E,4)
    u = obs @ W.T + b                          # (E,2)
    a = np.tanh(u)                             # (E,2)
    an = np.linalg.norm(a, axis=1, keepdims=True)
    a = a / np.maximum(1.0, an)                # ||a|| <= 1
    return a


#  Skóre: špatné vs opravené
# Bad score: +1 za každý krok v beacon zóně, epizoda končí dosažením goal
# Good score: průběžně -alpha*dist(goal) - beta a velký bonus za dosažení goal

alpha_dist = 0.05
beta_time = 0.01
terminal_bonus = 60.0

def simulate_batch(theta, starts, mode):
    """
    Vektorová simulace přes více startů.
    mode in {"bad","good"}.
    Vrací (mean_fitness, success_rate).
    success_rate = podíl epizod, které dosáhly goal.
    """
    pos = starts.astype(float).copy()   # (E,2)
    E = pos.shape[0]
    done = np.zeros(E, dtype=bool)      # dosažen cíl
    total = np.zeros(E, dtype=float)

    for _ in range(T):
        active = ~done
        if not np.any(active):
            break

        a = policy_action(theta, pos)
        pos_new = pos + step * a * active[:, None]

        d_goal = np.linalg.norm(pos_new - goal, axis=1)
        d_beac = np.linalg.norm(pos_new - beacon, axis=1)

        newly_done = active & (d_goal < goal_r)

        if mode == "bad":
            # Goodhart: optimalizujeme metrikou "čas v beacon zóně", ne skutečným cílem.
            total += (d_beac < beacon_r).astype(float) * active
            # žádný bonus za goal, pouze terminace, která dokonce snižuje možnost farmit beacon
        elif mode == "good":
            # Opravené skóre: hlavní je dosáhnout cíle a co nejdříve.
            total += (-alpha_dist * d_goal - beta_time) * active
            total += terminal_bonus * newly_done
        else:
            raise ValueError("mode musí být 'bad' nebo 'good'.")

        done = done | newly_done
        pos = pos_new

    return float(total.mean()), float(done.mean())

def simulate_episode_components(theta, start, mode):
    """
    Jedna epizoda pro vizualizaci a rozklad odměny.
    Vrací traj (L,2) a slovník komponent (délky L-1).
    """
    pos = np.array(start, dtype=float)
    traj = [pos.copy()]

    r_beac = []
    r_dist = []
    r_time = []
    r_term = []
    d_goal_list = []
    d_beac_list = []

    done = False
    for _ in range(T):
        if done:
            break

        a = policy_action(theta, pos[None, :])[0]
        pos_new = pos + step * a

        d_goal = float(np.linalg.norm(pos_new - goal))
        d_beac = float(np.linalg.norm(pos_new - beacon))

        newly_done = (d_goal < goal_r)

        if mode == "bad":
            rb = 1.0 if (d_beac < beacon_r) else 0.0
            r_beac.append(rb)
            r_dist.append(0.0)
            r_time.append(0.0)
            r_term.append(0.0)
        else:
            r_beac.append(0.0)
            r_dist.append(-alpha_dist * d_goal)
            r_time.append(-beta_time)
            r_term.append(terminal_bonus if newly_done else 0.0)

        d_goal_list.append(d_goal)
        d_beac_list.append(d_beac)

        done = newly_done
        pos = pos_new
        traj.append(pos.copy())

    comps = {
        "r_beacon": np.array(r_beac, dtype=float),
        "r_dist": np.array(r_dist, dtype=float),
        "r_time": np.array(r_time, dtype=float),
        "r_term": np.array(r_term, dtype=float),
        "d_goal": np.array(d_goal_list, dtype=float),
        "d_beac": np.array(d_beac_list, dtype=float),
    }
    return np.array(traj), comps


#  Evoluce: truncation selection + elitismus + Gaussian mutace

def run_evolution(mode, seed=0,
                  N=140, gens=70, elite_frac=0.20,
                  sigma0=0.9, sigma_decay=0.99,
                  starts_n=10, clip_val=8.0):
    rng = np.random.default_rng(seed)
    pop = rng.normal(0, 1, size=(N, 10))

    elite_n = max(1, int(N * elite_frac))
    sigma = sigma0

    best_hist = []
    avg_hist = []
    succ_hist = []
    best_params_hist = []

    for _ in range(gens):
        starts = sample_starts_ring(rng, starts_n)  # stejné starty pro všechny jedince v generaci

        fits = np.empty(N, dtype=float)
        succ = np.empty(N, dtype=float)
        for i in range(N):
            f, s = simulate_batch(pop[i], starts, mode)
            fits[i] = f
            succ[i] = s

        order = np.argsort(fits)[::-1]
        pop = pop[order]
        fits = fits[order]
        succ = succ[order]

        best_hist.append(fits[0])
        avg_hist.append(fits.mean())
        succ_hist.append(succ[0])
        best_params_hist.append(pop[0].copy())

        elites = pop[:elite_n]
        parents = elites[rng.integers(0, elite_n, size=N - elite_n)]
        offspring = parents + rng.normal(0, sigma, size=(N - elite_n, 10))
        offspring = np.clip(offspring, -clip_val, clip_val)

        pop = np.vstack([elites, offspring])
        sigma *= sigma_decay

    return (np.array(best_hist), np.array(avg_hist), np.array(succ_hist), np.array(best_params_hist))

# Spusťte oba běhy se stejným seedem pro srovnání
seed = 2
gens = 70

best_bad, avg_bad, succ_bad, bestP_bad = run_evolution("bad", seed=seed, gens=gens)
best_good, avg_good, succ_good, bestP_good = run_evolution("good", seed=seed, gens=gens)

print("KONEČNÝ VÝSLEDEK (bad score):  best fitness = %.3f, success(best) = %.3f" % (best_bad[-1], succ_bad[-1]))
print("KONEČNÝ VÝSLEDEK (good score): best fitness = %.3f, success(best) = %.3f" % (best_good[-1], succ_good[-1]))
print("Interpretace: 'success' měří původní cíl (dosažení goal), fitness se liší podle definice skóre.")


#  Learning curves: fitness a success rate

fig, ax = plt.subplots(2, 2, figsize=(12, 7))

ax[0,0].plot(best_bad, label="best")
ax[0,0].plot(avg_bad, label="average")
ax[0,0].set_title("Bad score: fitness v čase (optimalizuje beacon)")
ax[0,0].set_xlabel("generace")
ax[0,0].set_ylabel("fitness")
ax[0,0].legend()

ax[1,0].plot(succ_bad)
ax[1,0].set_title("Bad score: success rate nejlepšího jedince")
ax[1,0].set_xlabel("generace")
ax[1,0].set_ylabel("success (goal dosažen)")

ax[0,1].plot(best_good, label="best")
ax[0,1].plot(avg_good, label="average")
ax[0,1].set_title("Good score: fitness v čase (optimalizuje goal)")
ax[0,1].set_xlabel("generace")
ax[0,1].set_ylabel("fitness")
ax[0,1].legend()

ax[1,1].plot(succ_good)
ax[1,1].set_title("Good score: success rate nejlepšího jedince")
ax[1,1].set_xlabel("generace")
ax[1,1].set_ylabel("success (goal dosažen)")

plt.tight_layout()
plt.show()


#  Animace: trajektorie nejlepšího jedince napříč generacemi (bad vs good)

fixed_start = np.array([5.5, 0.0])

def setup_env_axes(ax, title):
    ax.set_aspect("equal", adjustable="box")
    ax.set_xlim(-7, 7)
    ax.set_ylim(-7, 7)
    ax.set_title(title)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.scatter([fixed_start[0]], [fixed_start[1]], s=40)  # start
    ax.scatter([goal[0]], [goal[1]], s=80)                # goal
    ax.scatter([beacon[0]], [beacon[1]], s=80)            # beacon
    ax.add_patch(plt.Circle((goal[0], goal[1]), goal_r, fill=False, linestyle="--"))
    ax.add_patch(plt.Circle((beacon[0], beacon[1]), beacon_r, fill=False, linestyle="--"))

fig, (axL, axR) = plt.subplots(1, 2, figsize=(12, 5))
setup_env_axes(axL, "Bad score: exploit (farmí beacon)")
setup_env_axes(axR, "Good score: správný cíl (jde do goal)")

(lineL,) = axL.plot([], [], linewidth=2)
(lineR,) = axR.plot([], [], linewidth=2)
txtL = axL.text(-6.8, 6.3, "")
txtR = axR.text(-6.8, 6.3, "")

def init():
    lineL.set_data([], [])
    lineR.set_data([], [])
    txtL.set_text("")
    txtR.set_text("")
    return lineL, lineR, txtL, txtR

def update(frame):
    trL, _ = simulate_episode_components(bestP_bad[frame], fixed_start, "bad")
    trR, _ = simulate_episode_components(bestP_good[frame], fixed_start, "good")

    lineL.set_data(trL[:,0], trL[:,1])
    lineR.set_data(trR[:,0], trR[:,1])

    txtL.set_text(f"gen {frame} | best fitness = {best_bad[frame]:.2f} | success = {succ_bad[frame]:.2f}")
    txtR.set_text(f"gen {frame} | best fitness = {best_good[frame]:.2f} | success = {succ_good[frame]:.2f}")

    return lineL, lineR, txtL, txtR

anim = animation.FuncAnimation(fig, update, frames=gens, init_func=init, interval=140, blit=True)
plt.close(fig)
display(HTML(anim.to_jshtml()))


#  Finální trajektorie z více startů (vizuální důkaz exploitu)

starts_demo = np.array([
    [ 6.0,  0.0],
    [ 0.0,  6.0],
    [-6.0,  0.0],
    [ 0.0, -6.0],
])

best_final_bad = bestP_bad[-1]
best_final_good = bestP_good[-1]

fig, (bx0, bx1) = plt.subplots(1, 2, figsize=(12, 5))
setup_env_axes(bx0, "Finální politika: bad score (typicky míří na beacon)")
setup_env_axes(bx1, "Finální politika: good score (typicky míří do goal)")

for s0 in starts_demo:
    tr0, _ = simulate_episode_components(best_final_bad, s0, "bad")
    bx0.plot(tr0[:,0], tr0[:,1])
    bx0.scatter([s0[0]], [s0[1]], s=25)

for s0 in starts_demo:
    tr1, _ = simulate_episode_components(best_final_good, s0, "good")
    bx1.plot(tr1[:,0], tr1[:,1])
    bx1.scatter([s0[0]], [s0[1]], s=25)

plt.tight_layout()
plt.show()


#  Rozklad skóre v čase: proč je to exploit

traj_bad, comp_bad = simulate_episode_components(best_final_bad, fixed_start, "bad")
traj_good, comp_good = simulate_episode_components(best_final_good, fixed_start, "good")

def cumulative_plot(comps, title):
    # kumulativní součty jednotlivých složek a celku
    rb = np.cumsum(comps["r_beacon"]) if len(comps["r_beacon"]) else np.array([0.0])
    rd = np.cumsum(comps["r_dist"]) if len(comps["r_dist"]) else np.array([0.0])
    rt = np.cumsum(comps["r_time"]) if len(comps["r_time"]) else np.array([0.0])
    rT = np.cumsum(comps["r_term"]) if len(comps["r_term"]) else np.array([0.0])
    total = rb + rd + rt + rT

    plt.figure(figsize=(8, 4))
    plt.plot(total, label="cumulative total")
    if np.any(comps["r_beacon"] != 0): plt.plot(rb, label="cumulative beacon")
    if np.any(comps["r_dist"] != 0):   plt.plot(rd, label="cumulative dist penalty")
    if np.any(comps["r_time"] != 0):   plt.plot(rt, label="cumulative time penalty")
    if np.any(comps["r_term"] != 0):   plt.plot(rT, label="cumulative terminal bonus")
    plt.title(title)
    plt.xlabel("krok")
    plt.ylabel("kumulativní odměna")
    plt.legend()
    plt.show()

cumulative_plot(comp_bad, "Bad score: kumulativní odměna (odměňuje pobyt u beacon)")
cumulative_plot(comp_good, "Good score: kumulativní odměna (bonus za goal + shaping)")

plt.figure(figsize=(12, 4))
plt.plot(comp_bad["d_goal"], label="bad: dist(goal)")
plt.plot(comp_bad["d_beac"], label="bad: dist(beacon)")
plt.plot(comp_good["d_goal"], label="good: dist(goal)")
plt.plot(comp_good["d_beac"], label="good: dist(beacon)")
plt.title("Diagnostika: vzdálenosti v čase (fixed_start)")
plt.xlabel("krok")
plt.ylabel("vzdálenost")
plt.legend()
plt.show()


#  Test success rate na větším vzorku startů (nezávislá evaluace)

rng_test = np.random.default_rng(123)
test_starts = sample_starts_ring(rng_test, 300)

_, sr_bad = simulate_batch(best_final_bad, test_starts, "bad")   # success vždy měří dosažení goal
_, sr_good = simulate_batch(best_final_good, test_starts, "good")

print("TEST success rate (300 startů):")
print("  finální bad score  :", round(sr_bad, 3))
print("  finální good score :", round(sr_good, 3))


# Hyperparam sweep (jak to prakticky ladit a diagnostikovat).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time


# Hyperparam sweep: jak ladit evoluční algoritmus a jak diagnostikovat
# Prostředí: 2D fitness krajina (Rastrigin), maximalizujeme fitness = -Rastrigin
# Sledujeme: best fitness, average fitness, diverzitu populace a stabilitu přes více seedů

#  Fitness krajina

def rastrigin(x, y):
    return 20 + (x**2 - 10*np.cos(2*np.pi*x)) + (y**2 - 10*np.cos(2*np.pi*y))

def fitness_xy(pop_xy):
    x = pop_xy[:, 0]
    y = pop_xy[:, 1]
    return -rastrigin(x, y)  # maximum je 0 v (0,0)


#  Evoluce: truncation selection + elitismus + Gaussian mutace

def evolve_once(
    N=120,
    gens=70,
    elite_frac=0.2,
    sigma0=0.6,
    sigma_decay=0.99,
    bounds=(-5.12, 5.12),
    seed=0
):
    rng = np.random.default_rng(seed)
    lo, hi = bounds

    elite_n = max(1, int(N * elite_frac))
    sigma = sigma0

    # start: uniformně v doméně
    pop = rng.uniform(lo, hi, size=(N, 2))

    best_hist = np.empty(gens, dtype=float)
    avg_hist  = np.empty(gens, dtype=float)
    div_hist  = np.empty(gens, dtype=float)  # jednoduchá diverzita = průměr směrodatných odchylek v x,y

    best_xy_hist = np.empty((gens, 2), dtype=float)

    for g in range(gens):
        fit = fitness_xy(pop)

        order = np.argsort(fit)[::-1]
        pop = pop[order]
        fit = fit[order]

        best_hist[g] = fit[0]
        avg_hist[g]  = fit.mean()
        div_hist[g]  = pop.std(axis=0).mean()
        best_xy_hist[g] = pop[0]

        elites = pop[:elite_n]
        parents = elites[rng.integers(0, elite_n, size=N - elite_n)]

        offspring = parents + rng.normal(0, sigma, size=(N - elite_n, 2))
        offspring[:, 0] = np.clip(offspring[:, 0], lo, hi)
        offspring[:, 1] = np.clip(offspring[:, 1], lo, hi)

        pop = np.vstack([elites, offspring])
        sigma *= sigma_decay

    return {
        "best": best_hist,
        "avg": avg_hist,
        "div": div_hist,
        "best_xy": best_xy_hist,
        "final_best": float(best_hist[-1]),
        "final_xy": best_xy_hist[-1].copy(),
    }


#  Sweep runner (více seedů pro stabilitu)

def run_sweep(param_name, param_values, base_cfg, seeds):
    out = {}
    for v in param_values:
        cfg = dict(base_cfg)
        cfg[param_name] = v
        runs = []
        for s in seeds:
            cfg["seed"] = s
            runs.append(evolve_once(**cfg))
        out[v] = runs
    return out

def summarize_sweep(results, near_opt_threshold=-1.0):
    rows = []
    for k in sorted(results.keys()):
        finals = np.array([r["final_best"] for r in results[k]], dtype=float)
        rows.append((
            k,
            finals.mean(),
            finals.std(ddof=1) if len(finals) > 1 else 0.0,
            finals.max(),
            (finals > near_opt_threshold).mean()
        ))
    return rows

def print_summary(title, rows, unit=""):
    print(title)
    print("param | mean(final_best) | sd | best(run) | P(final_best > %.2f)" % (-1.0))
    for (k, m, sd, mx, p) in rows:
        if unit:
            print(f"{k}{unit:>0} | {m:>14.3f} | {sd:>6.3f} | {mx:>9.3f} | {p:>16.3f}")
        else:
            print(f"{k:>5} | {m:>14.3f} | {sd:>6.3f} | {mx:>9.3f} | {p:>16.3f}")
    print()


#  Plot helper: průměr + IQR přes seedy

def plot_band(ax, x, y_mat, label):
    mean = y_mat.mean(axis=0)
    p25, p75 = np.percentile(y_mat, [25, 75], axis=0)
    line, = ax.plot(x, mean, label=label)
    ax.fill_between(x, p25, p75, alpha=0.20, color=line.get_color())

def plot_sweep_curves(results, metric_key, title, xlabel="generace", ylabel=""):
    fig, ax = plt.subplots(figsize=(9, 4))
    x = None
    for k in sorted(results.keys()):
        mat = np.vstack([r[metric_key] for r in results[k]])
        if x is None:
            x = np.arange(mat.shape[1])
        plot_band(ax, x, mat, label=f"{k}")
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel if ylabel else metric_key)
    ax.legend(title="param", ncol=2)
    plt.tight_layout()
    plt.show()

def boxplot_final(results, title, ylabel="final best fitness"):
    keys = sorted(results.keys())
    data = [np.array([r["final_best"] for r in results[k]]) for k in keys]
    plt.figure(figsize=(9, 4))
    plt.boxplot(data, labels=[str(k) for k in keys], showmeans=True)
    plt.title(title)
    plt.xlabel("param")
    plt.ylabel(ylabel)
    plt.tight_layout()
    plt.show()


#  Nastavení běhu

base_cfg = dict(
    N=120,
    gens=70,
    elite_frac=0.2,
    sigma0=0.6,
    sigma_decay=0.99,
    bounds=(-5.12, 5.12),
)

seeds = list(range(6))  # více seedů = lepší diagnostika stability


#  Sweep A: síla mutace sigma0 (exploration vs exploitation)

sigma_values = [0.05, 0.20, 0.60, 1.20]

t0 = time.time()
res_sigma = run_sweep("sigma0", sigma_values, base_cfg, seeds)
t1 = time.time()

rows_sigma = summarize_sweep(res_sigma, near_opt_threshold=-1.0)
print_summary("SWEEP A: sigma0 (počáteční síla mutace)", rows_sigma)
print("Čas sweep A:", round(t1 - t0, 2), "s\n")

plot_sweep_curves(
    res_sigma,
    metric_key="best",
    title="SWEEP A (sigma0): best fitness v čase (průměr + IQR přes seedy)",
    ylabel="best fitness (max 0)"
)
plot_sweep_curves(
    res_sigma,
    metric_key="div",
    title="SWEEP A (sigma0): diverzita populace v čase (průměr + IQR)",
    ylabel="diverzita (průměr sd v x,y)"
)
boxplot_final(
    res_sigma,
    title="SWEEP A (sigma0): rozdělení final best fitness přes seedy",
    ylabel="final best fitness (blíže k 0 je lepší)"
)


#  Sweep B: velikost populace N (šum vs výpočetní náklady)

N_values = [20, 60, 120, 240]

t0 = time.time()
res_N = run_sweep("N", N_values, base_cfg, seeds)
t1 = time.time()

rows_N = summarize_sweep(res_N, near_opt_threshold=-1.0)
print_summary("SWEEP B: N (velikost populace)", rows_N)
print("Čas sweep B:", round(t1 - t0, 2), "s\n")

plot_sweep_curves(
    res_N,
    metric_key="best",
    title="SWEEP B (N): best fitness v čase (průměr + IQR přes seedy)",
    ylabel="best fitness (max 0)"
)
plot_sweep_curves(
    res_N,
    metric_key="avg",
    title="SWEEP B (N): average fitness v čase (průměr + IQR přes seedy)",
    ylabel="average fitness"
)
boxplot_final(
    res_N,
    title="SWEEP B (N): rozdělení final best fitness přes seedy",
    ylabel="final best fitness (blíže k 0 je lepší)"
)


#  Sweep C: elitismus (elite_frac) jako selekční tlak

elite_values = [0.05, 0.20, 0.50]

t0 = time.time()
res_elite = run_sweep("elite_frac", elite_values, base_cfg, seeds)
t1 = time.time()

rows_elite = summarize_sweep(res_elite, near_opt_threshold=-1.0)
print_summary("SWEEP C: elite_frac (podíl elit)", rows_elite)
print("Čas sweep C:", round(t1 - t0, 2), "s\n")

plot_sweep_curves(
    res_elite,
    metric_key="best",
    title="SWEEP C (elite_frac): best fitness v čase (průměr + IQR přes seedy)",
    ylabel="best fitness (max 0)"
)
plot_sweep_curves(
    res_elite,
    metric_key="div",
    title="SWEEP C (elite_frac): diverzita populace v čase (průměr + IQR)",
    ylabel="diverzita (průměr sd v x,y)"
)
boxplot_final(
    res_elite,
    title="SWEEP C (elite_frac): rozdělení final best fitness přes seedy",
    ylabel="final best fitness (blíže k 0 je lepší)"
)


#  Krátká diagnostická poznámka v číslech (stagnace v posledních generacích)

def stagnation_index(best_hist, window=10):
    # průměrné zlepšení v posledním okně generací (blíže k 0 je lepší, takže sledujeme rozdíl)
    if len(best_hist) <= window:
        return np.nan
    return float(best_hist[-1] - best_hist[-window])

def print_stagnation(title, results):
    print(title)
    for k in sorted(results.keys()):
        st = np.array([stagnation_index(r["best"], window=10) for r in results[k]])
        print(f"param={k}: mean last-10 improvement = {st.mean():.4f} (kladné je zlepšení), sd = {st.std(ddof=1) if len(st)>1 else 0.0:.4f}")
    print()

print_stagnation("DIAGNOSTIKA: průměrné zlepšení best fitness v posledních 10 generacích (sweep A)", res_sigma)
print_stagnation("DIAGNOSTIKA: průměrné zlepšení best fitness v posledních 10 generacích (sweep B)", res_N)
print_stagnation("DIAGNOSTIKA: průměrné zlepšení best fitness v posledních 10 generacích (sweep C)", res_elite)
